In [1]:
import hashlib  # For computing SHA-256 hashes
import time     # To timestamp blocks

In Bitcoin and many similar blockchains, the mining difficulty target is a 256-bit number. Storing this full 256-bit target in every block header would be inefficient, so a compact form called “bits” is used. This 32-bit (4-byte) field encodes the target using a floating‐point–like format:

Exponent (1 byte):
The most significant byte represents the exponent. It indicates how many bytes the full target takes up when expressed in big-endian format.

Coefficient (3 bytes):
The remaining three bytes represent the coefficient (or mantissa).

The full target is then computed as:

  target = coefficient × 2^(8 × (exponent – 3))

When mining, a block’s hash is interpreted as a large integer. A block is considered valid if its hash is lower than this target value.

In [2]:
def bits_to_target(bits):
    """
    Convert the compact representation of the target (bits) to the full target value.
    
    The 'bits' value is a 32-bit unsigned integer:
      - The most significant byte is the exponent.
      - The remaining 3 bytes are the coefficient.
    
    The target is computed as:
         target = coefficient * 2^(8*(exponent - 3))
    """
    exponent = bits >> 24             # Extract the exponent (top 8 bits)
    coefficient = bits & 0xffffff     # Extract the coefficient (lower 24 bits)
    target = coefficient * (1 << (8 * (exponent - 3)))
    return target

# Example: Print target from a given bits value (commonly used in test networks)
bits_example = 0x1e0ffff0
target_example = bits_to_target(bits_example)
print("Example bits:", hex(bits_example))
print("Computed target:", hex(target_example))


Example bits: 0x1e0ffff0
Computed target: 0xffff0000000000000000000000000000000000000000000000000000000


In [3]:
class Block:
    def __init__(self, index, previous_hash, data):
        self.index = index                        # Position of the block in the chain
        self.timestamp = time.time()              # Time when block is created
        self.data = data                          # Block data (e.g., transactions)
        self.previous_hash = previous_hash        # Hash of the previous block in the chain
        self.nonce = 0                            # Value to be adjusted during mining
        self.hash = self.compute_hash()           # Initial hash based on current values

    def compute_hash(self):
        # Concatenate block attributes into a string and compute its SHA-256 hash.
        block_content = f"{self.index}{self.timestamp}{self.data}{self.previous_hash}{self.nonce}"
        return hashlib.sha256(block_content.encode()).hexdigest()

In [4]:
def mine_block(block, bits=0x1e0ffff0, verbose=True):
    """
    Mines a block by finding a nonce such that the block's hash (interpreted as an integer)
    is lower than the target computed from the 'bits' value.
    
    Parameters:
      - block: The Block object to be mined.
      - bits: The compact representation of the target (default provided is 0x1e0ffff0).
      - verbose: If True, prints detailed debug info during mining.
      
    Returns:
      The valid nonce and hash when the block is successfully mined.
    """
    target = bits_to_target(bits)
    if verbose:
        print(f"\nStarting mining for Block {block.index} with target: {hex(target)}")
    iteration = 0
    # Loop until the block's hash, converted to an integer, is lower than the target.
    while int(block.hash, 16) >= target:
        block.nonce += 1
        block.hash = block.compute_hash()
        iteration += 1
        # For demonstration, print every 1000 iterations to avoid excessive output.
        if verbose and iteration % 1000 == 0:
            print(f"Iteration {iteration}: nonce = {block.nonce}, hash = {block.hash}")
    if verbose:
        print(f"Block {block.index} mined after {iteration} iterations!")
        print(f"Final nonce: {block.nonce}, Final hash: {block.hash}\n")
    return block.nonce, block.hash

In [5]:
class Blockchain:
    def __init__(self):
        self.chain = []             # List to store blocks in the blockchain
        self.bits = 0x1e0ffff0      # The compact "bits" representation for mining target
        self.create_genesis_block()

    def create_genesis_block(self):
        # Create the first block with default previous hash "0"
        genesis_block = Block(0, "0", "Genesis Block")
        mine_block(genesis_block, self.bits, verbose=True)
        self.chain.append(genesis_block)

    def add_block(self, data):
        # Create a new block using the hash of the latest block as its previous_hash
        last_block = self.chain[-1]
        new_block = Block(len(self.chain), last_block.hash, data)
        mine_block(new_block, self.bits, verbose=True)
        self.chain.append(new_block)

    def display_chain(self):
        # Print detailed information for each block in the blockchain.
        for block in self.chain:
            print(f"Index: {block.index}")
            print(f"Timestamp: {block.timestamp}")
            print(f"Data: {block.data}")
            print(f"Previous Hash: {block.previous_hash}")
            print(f"Nonce: {block.nonce}")
            print(f"Hash: {block.hash}")
            print("-" * 50)


In [6]:
# Initialize the blockchain (genesis block is created and mined automatically)
blockchain = Blockchain()

# Display details of the genesis block
print("Genesis Block:")
genesis = blockchain.chain[0]
print(f"Index: {genesis.index}")
print(f"Timestamp: {genesis.timestamp}")
print(f"Data: {genesis.data}")
print(f"Previous Hash: {genesis.previous_hash}")
print(f"Nonce: {genesis.nonce}")
print(f"Hash: {genesis.hash}")
print("-" * 50)


Starting mining for Block 0 with target: 0xffff0000000000000000000000000000000000000000000000000000000
Iteration 1000: nonce = 1000, hash = 938bcc67245030a1600323580a78f7b841499713aa100134e981626e83b9a420
Iteration 2000: nonce = 2000, hash = 0c9c1f6210e288872e24021de0433db083a4e738bb018461e403dd31314f9e9c
Iteration 3000: nonce = 3000, hash = decaf4c1aac06bc0567ce029d0b744dcc65444b4c85d235818b33e88d41ee163
Iteration 4000: nonce = 4000, hash = 89ab825721b8b90d6b25e4b6ec5efd5ae875e22f0e635f1535bcf64fab46b758
Iteration 5000: nonce = 5000, hash = 3a03b88017b3291bc18ab01eb1d2d24489b4ea0c8df2d2ac8928120def02ab4b
Iteration 6000: nonce = 6000, hash = 63c6a28cddf9cf6f88ce52fbfaa3877cf2551370109c25e1440d19d5e0f37ec5
Iteration 7000: nonce = 7000, hash = 2f8f6a7b511de6d89bbce52bcdedde8a791ee7798aa807e3d7da1ac9213799a3
Iteration 8000: nonce = 8000, hash = f4a8ef1dacae943b3fb8e0b53b9f3d9b4a8965411cd28a98ec6439884c0aec24
Iteration 9000: nonce = 9000, hash = fe85b52a6a276d5cd17f484666aed8f503d75b29c66

In [7]:
# Add the first block with sample transaction data.
blockchain.add_block("Transaction A to B: $50")
print("Added Block 1:")

# Display details of the newly added block.
block1 = blockchain.chain[-1]
print(f"Index: {block1.index}")
print(f"Timestamp: {block1.timestamp}")
print(f"Data: {block1.data}")
print(f"Previous Hash: {block1.previous_hash}")
print(f"Nonce: {block1.nonce}")
print(f"Hash: {block1.hash}")
print("-" * 50)

# Add the second block.
blockchain.add_block("Transaction C to D: $20")
print("Added Block 2:")
block2 = blockchain.chain[-1]
print(f"Index: {block2.index}")
print(f"Timestamp: {block2.timestamp}")
print(f"Data: {block2.data}")
print(f"Previous Hash: {block2.previous_hash}")
print(f"Nonce: {block2.nonce}")
print(f"Hash: {block2.hash}")
print("-" * 50)

# Add the third block.
blockchain.add_block("Transaction E to F: $100")
print("Added Block 3:")
block3 = blockchain.chain[-1]
print(f"Index: {block3.index}")
print(f"Timestamp: {block3.timestamp}")
print(f"Data: {block3.data}")
print(f"Previous Hash: {block3.previous_hash}")
print(f"Nonce: {block3.nonce}")
print(f"Hash: {block3.hash}")
print("-" * 50)


Starting mining for Block 1 with target: 0xffff0000000000000000000000000000000000000000000000000000000
Iteration 1000: nonce = 1000, hash = f3afa4e41851b04a2047ed19dc13ab9d7e0a50e9dc594b721113b8cfe1a96404
Iteration 2000: nonce = 2000, hash = f0206b72b7755a8a06dd020f960cdce686cb2841a4e4436ad30ad39d38c19ccf
Iteration 3000: nonce = 3000, hash = 55378587c6417f41742ca29a60552a927b801157d9fc31257bda651a6d44c680
Iteration 4000: nonce = 4000, hash = 450a183f21f6eb7373cddc1dcfaf52762436cc69fcd94d109b6fda0abba120e6
Iteration 5000: nonce = 5000, hash = 048266b23e6e16bcf51bc00e313cc69f6f1ba37b7d36c2aea20e7bf563005b77
Iteration 6000: nonce = 6000, hash = 364ce84426ab73968cdee3a96342a81db9b7c6f543752e9c56604d78317600b7
Iteration 7000: nonce = 7000, hash = b9f213e56e9003f5d98581e1f596ffcf86a74287bf28d86f831bf460e9823fe5
Iteration 8000: nonce = 8000, hash = c5d0a14d49a7261c36fd2defc39b3f81866fc74ee78ae801c35b9c53a6b4cad8
Iteration 9000: nonce = 9000, hash = 21c966e780142986e3c5cd85f177fc3d337baf681e2

In [8]:
# Print detailed information for every block in the blockchain.
print("Complete Blockchain Details:")
blockchain.display_chain()

Complete Blockchain Details:
Index: 0
Timestamp: 1742829912.5727808
Data: Genesis Block
Previous Hash: 0
Nonce: 245022
Hash: 000007b53e47c6f6fea7e8c1b1cc932a9d58f827cfbb1fcc2753aaedbf15118d
--------------------------------------------------
Index: 1
Timestamp: 1742829913.111326
Data: Transaction A to B: $50
Previous Hash: 000007b53e47c6f6fea7e8c1b1cc932a9d58f827cfbb1fcc2753aaedbf15118d
Nonce: 777041
Hash: 000001b4f22f0f038bda203ecd2bd8008c93083e5a6107ba0162e7a6544b5a4f
--------------------------------------------------
Index: 2
Timestamp: 1742829914.7940602
Data: Transaction C to D: $20
Previous Hash: 000001b4f22f0f038bda203ecd2bd8008c93083e5a6107ba0162e7a6544b5a4f
Nonce: 308670
Hash: 00000f49c6700feb18fe97a3d7ce37494cb1b2d1b0f1e0b5f42e360fa462cba1
--------------------------------------------------
Index: 3
Timestamp: 1742829915.5949755
Data: Transaction E to F: $100
Previous Hash: 00000f49c6700feb18fe97a3d7ce37494cb1b2d1b0f1e0b5f42e360fa462cba1
Nonce: 48708
Hash: 00000a208df898ff17294

In [9]:
# Finally, display the hash of every block in the blockchain.
print("Hashes of all blocks in the blockchain:")
for block in blockchain.chain:
    print(f"Block {block.index} hash: {block.hash}")

Hashes of all blocks in the blockchain:
Block 0 hash: 000007b53e47c6f6fea7e8c1b1cc932a9d58f827cfbb1fcc2753aaedbf15118d
Block 1 hash: 000001b4f22f0f038bda203ecd2bd8008c93083e5a6107ba0162e7a6544b5a4f
Block 2 hash: 00000f49c6700feb18fe97a3d7ce37494cb1b2d1b0f1e0b5f42e360fa462cba1
Block 3 hash: 00000a208df898ff17294387b86881a1e518c534aea7a381f603f4c4f72d59bf


In [10]:
print("Starting step-by-step blockchain verification...\n")

# Iterate through the blockchain starting from the second block.
for i in range(1, len(blockchain.chain)):
    current = blockchain.chain[i]
    previous = blockchain.chain[i - 1]
    
    print(f"Verifying Block {current.index}:")
    
    # Recompute the current block's hash.
    recomputed_hash = current.compute_hash()
    print(f"  Stored hash:    {current.hash}")
    print(f"  Recomputed hash:{recomputed_hash}")
    
    if current.hash == recomputed_hash:
        print("  [OK] Block hash is valid.")
    else:
        print("  [ERROR] Block hash mismatch!")
    
    # Check that the current block correctly references the previous block's hash.
    print(f"  Previous hash in block: {current.previous_hash}")
    print(f"  Hash of previous block: {previous.hash}")
    
    if current.previous_hash == previous.hash:
        print("  [OK] Previous hash is valid.")
    else:
        print("  [ERROR] Previous hash mismatch!")
    
    print("-" * 50)

print("Blockchain verification complete!")

Starting step-by-step blockchain verification...

Verifying Block 1:
  Stored hash:    000001b4f22f0f038bda203ecd2bd8008c93083e5a6107ba0162e7a6544b5a4f
  Recomputed hash:000001b4f22f0f038bda203ecd2bd8008c93083e5a6107ba0162e7a6544b5a4f
  [OK] Block hash is valid.
  Previous hash in block: 000007b53e47c6f6fea7e8c1b1cc932a9d58f827cfbb1fcc2753aaedbf15118d
  Hash of previous block: 000007b53e47c6f6fea7e8c1b1cc932a9d58f827cfbb1fcc2753aaedbf15118d
  [OK] Previous hash is valid.
--------------------------------------------------
Verifying Block 2:
  Stored hash:    00000f49c6700feb18fe97a3d7ce37494cb1b2d1b0f1e0b5f42e360fa462cba1
  Recomputed hash:00000f49c6700feb18fe97a3d7ce37494cb1b2d1b0f1e0b5f42e360fa462cba1
  [OK] Block hash is valid.
  Previous hash in block: 000001b4f22f0f038bda203ecd2bd8008c93083e5a6107ba0162e7a6544b5a4f
  Hash of previous block: 000001b4f22f0f038bda203ecd2bd8008c93083e5a6107ba0162e7a6544b5a4f
  [OK] Previous hash is valid.
-----------------------------------------------